In [1]:
from twelvedata import TDClient
import pprint
import pandas as pd
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import sqlite3
td = TDClient(apikey="faa93033c87f4b8290b9486920e97a86")

In [2]:
# Cryptocurrencies List, td.get_cryptocurrencies_list() see at: (https://twelvedata.com/blog/get-high-quality-financial-data-directly-into-python)
all_Cryptos = td.get_cryptocurrencies_list().as_json()

#Print firtst item in list as visual aid
all_Cryptos[0]

{'symbol': '0xBTC/BTC',
 'available_exchanges': ['Hotbit', 'Mercatox'],
 'currency_base': '0xBitcoin',
 'currency_quote': 'Bitcoin'}

In [3]:
# Filter the symbols that are being compared to USD
usd_symbols = [crypto['symbol'] for crypto in all_Cryptos if '/USD' in crypto['symbol']]
print("Number of USD-paired symbols:", len(usd_symbols))

Number of USD-paired symbols: 1641


In [4]:
ts = td.time_series(
    symbol="BTC/USD",
    timezone="America/New_York",
    interval="30min",
    outputsize=5
).as_pandas()
ts

,open,high,low,close
datetime,,,,
2023-07-21 20:00:00,29907.15,29936.85,29903.56,29931.75
2023-07-21 19:30:00,29930.05,29936.69,29900.38,29907.98
2023-07-21 19:00:00,29932.45,29941.88,29895.87,29930.01
2023-07-21 18:30:00,29936.03,29966.05,29926.39,29932.44
2023-07-21 18:00:00,29904.88,29951.70,29893.01,29936.03


In [5]:
ts = td.time_series(
    symbol="BTC/USD",
    interval="1month",
    start_date="2018-07-20",
    end_date="2023-07-20"
).as_pandas()
ts

,open,high,low,close
datetime,,,,
2023-07-01,30466.73000,31862.21000,29525.00000,29931.75000
2023-06-01,27221.54000,31443.67000,24808.97000,30466.72000
2023-05-01,29240.49000,29850.00000,25810.00000,27223.08000
2023-04-01,28477.10938,31050.00000,26965.14000,29247.14000
2023-03-01,23135.34961,29168.71094,19582.88086,28477.10938
2023-02-01,23128.68945,25235.83984,21415.03906,23128.75977
2023-01-01,16528.02930,23944.60938,16492.97070,23127.67969
2022-12-01,17167.33008,18360.83984,16319.29980,16528.02930
2022-11-01,20492.08984,21470.58008,15500.33984,17167.33984


In [27]:
# Create a list to store the DataFrames for each symbol in the batch
dfs = []

# Loop through batches of symbols
symbol_batch = ['BTC/USD', 'ETH/USD', 'USDT/USD', 'XRP/USD', 'BNB/USD', 'ADA/USD', 'SOL/USD', 'DOGE/USD']

# Process each symbol in the current batch
for symbol in symbol_batch:
    try:
        ts = td.time_series(
            symbol=symbol,
            interval="1week",
            timezone="America/New_York",
            outputsize=208
        ).with_bbands(time_period=3).with_macd(fast_period=4, slow_period=8, signal_period=2).with_rsi(time_period=6).as_pandas()

        # Rename the 'close' column to the symbol name for clarity
        ts.rename(columns={'close': symbol}, inplace=True)

        # Add a new numeric index column without dropping the original timestamp index
        ts.reset_index(inplace=True)

        # Add the DataFrame for the current symbol to the list
        dfs.append(ts)

    except Exception as e:
        print(f"Error fetching data for symbol '{symbol}': {str(e)}")

# Introduce a delay of 1 minute between API calls
time.sleep(60)

print("Total DataFrames fetched:", len(dfs))

Total DataFrames fetched: 8


In [32]:
dfs[7]

,datetime,open,high,low,DOGE/USD,upper_band,middle_band,lower_band,macd,macd_signal,macd_hist,rsi
0,2023-07-17,0.06955,0.07684,0.06704,0.07256,0.07506,0.06917,0.06328,0.00022,-0.00026,0.00048,57.16301
1,2023-07-10,0.06539,0.07517,0.06333,0.06956,0.07110,0.06765,0.06420,-0.00084,-0.00123,0.00038,49.81943
2,2023-07-03,0.06800,0.07189,0.06408,0.06538,0.06885,0.06671,0.06457,-0.00182,-0.00200,0.00018,37.34868
3,2023-06-26,0.06676,0.07138,0.06006,0.06800,0.07069,0.06562,0.06054,-0.00197,-0.00235,0.00039,42.92004
4,2023-06-19,0.06209,0.07015,0.06070,0.06675,0.06813,0.06346,0.05879,-0.00291,-0.00313,0.00022,39.32130
...,...,...,...,...,...,...,...,...,...,...,...,...
92,2021-10-11,0.22880,0.24550,0.21640,0.23770,0.24238,0.22943,0.21649,0.00000,0.00000,0.00000,0.00000
93,2021-10-04,0.22180,0.27180,0.21260,0.22870,0.23835,0.21860,0.19885,0.00000,0.00000,0.00000,0.00000
94,2021-09-27,0.20510,0.22660,0.19480,0.22190,0.24278,0.22000,0.19722,0.00000,0.00000,0.00000,0.00000
95,2021-09-20,0.23290,0.23350,0.19430,0.20520,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [10]:
dfs[0]

,open,high,low,0xBTC/USD
datetime,,,,
2023-07-01,0.09097,0.09324,0.08513,0.08807
2023-06-01,0.10860,0.11398,0.07698,0.09091
2023-05-01,0.22209,0.22539,0.09500,0.10856
2023-04-01,0.22895,0.29980,0.17010,0.22292
2023-03-01,0.20290,0.25479,0.17339,0.22893
2023-02-01,0.20294,0.21891,0.18008,0.20278
2023-01-01,0.15744,0.21398,0.15241,0.20304
2022-12-01,0.16426,0.20333,0.15327,0.15741
2022-11-01,0.19030,0.20078,0.12969,0.16435


In [29]:
# Create a database connection using SQLAlchemy
engine = create_engine('sqlite:///crypto_data.db')

# Loop through the DataFrames in dfs and save each DataFrame to the database
for df in dfs:
    df.to_sql(name='crypto_data', con=engine, if_exists='append', index=False)

# Close the database connection
engine.dispose()

      open     high      low  0xBTC/USD
0  0.09097  0.09324  0.08513    0.08784
1  0.10860  0.11398  0.07698    0.09091
2  0.22209  0.22539  0.09500    0.10856


,open,high,low,0xBTC/USD,open,high,low,0xBTC/USD,open,high,...,low,BAKE/USD,open,high,low,BAKE/USD,open,high,low,BAKE/USD
0,0.09097,0.09324,0.08513,0.08784,0.22895,0.29980,0.17010,0.22292,0.15744,0.21398,...,1.6770,2.003,1.9373,2.4475,1.4474,2.0143,6.0199,7.9111,1.8769,4.1461
1,0.10860,0.11398,0.07698,0.09091,0.20290,0.25479,0.17339,0.22893,0.16426,0.20333,...,1.4070,1.716,4.1350,4.5929,1.5707,1.9349,NaN,NaN,NaN,NaN
2,0.22209,0.22539,0.09500,0.10856,0.20294,0.21891,0.18008,0.20278,0.19030,0.20078,...,1.8861,2.634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create a SQLite database and establish a connection
conn = sqlite3.connect('crypto_data.db')

# Loop through the DataFrames in dfs and save each DataFrame to the database
for df in dfs:
    df.to_sql(name='crypto_data', con=conn, if_exists='append', index=False)

# Close the database connection
conn.close()

In [30]:
def get_stock_data(symbol):
    try:
        td = TDClient(apikey="faa93033c87f4b8290b9486920e97a86")
        response = td.time_series(
            symbol=symbol[1],
            interval="30min",
            outputsize=50
            )
        
        if response:
            data = response.as_json()
            return data
        else:
            print("No data received from the API.")
            return None
            
    except Exception as e:
        print(f"Error occurred during the API call: {e}")
        return None

# Example usage for AAPL stock
symbol = "BTC/USD"
data = get_stock_data(symbol)

if data:
    pprint.pprint("API call successful!")
    pprint.pprint(data)

'API call successful!'
({'close': '14.65750',
  'datetime': '2023-07-20 15:30:00',
  'high': '14.66000',
  'low': '14.58000',
  'open': '14.59500',
  'volume': '10401770'},
 {'close': '14.59500',
  'datetime': '2023-07-20 15:00:00',
  'high': '14.60500',
  'low': '14.54000',
  'open': '14.54000',
  'volume': '4860082'},
 {'close': '14.54270',
  'datetime': '2023-07-20 14:30:00',
  'high': '14.59000',
  'low': '14.54000',
  'open': '14.56500',
  'volume': '2313557'},
 {'close': '14.56500',
  'datetime': '2023-07-20 14:00:00',
  'high': '14.61000',
  'low': '14.55000',
  'open': '14.61000',
  'volume': '2015613'},
 {'close': '14.61000',
  'datetime': '2023-07-20 13:30:00',
  'high': '14.65000',
  'low': '14.59000',
  'open': '14.65000',
  'volume': '2973305'},
 {'close': '14.64520',
  'datetime': '2023-07-20 13:00:00',
  'high': '14.65000',
  'low': '14.61000',
  'open': '14.62000',
  'volume': '2412057'},
 {'close': '14.62000',
  'datetime': '2023-07-20 12:30:00',
  'high': '14.64000',
